In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import gym
from main.alpaca import *
from main.datagen import *
from main.dataViz import *
import main.train_policy
from baselines.common import tf_util as U
from baselines import logger

/Users/james/.pyenv/versions/3.5.2/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [2]:
cfg_filename = 'configs/hopper-config.yml'
with open(cfg_filename,'r') as ymlfile:
    config = yaml.load(ymlfile)

In [3]:
env_id = 'Hopper-v2'
env = gym.make(env_id)
logger.configure()
pi = train_policy.train(env_id, num_timesteps=1, seed=1124)
U.load_state('policies/hopper')

def train_hopper_pol(ob):
    return pi.act(stochastic=True, ob=ob)[0]
def test_hopper_pol(ob):
    return pi.act(stochastic=False, ob=ob)[0]

running build_ext
building 'mujoco_py.cymj' extension
/usr/local/bin/gcc-7 -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -DONMAC -I/Users/james/Dropbox/code/sysid-nets/si2_env/lib/python3.5/site-packages/mujoco_py -I/Users/james/.mujoco/mjpro150/include -I/Users/james/Dropbox/code/sysid-nets/si2_env/lib/python3.5/site-packages/numpy/core/include -I/Users/james/Dropbox/code/sysid-nets/si2_env/include -I/Users/james/.pyenv/versions/3.5.2/include/python3.5m -c /Users/james/Dropbox/code/sysid-nets/si2_env/lib/python3.5/site-packages/mujoco_py/cymj.c -o /Users/james/Dropbox/code/sysid-nets/si2_env/lib/python3.5/site-packages/mujoco_py/generated/_pyxbld_1.50.1.56_35_macextensionbuilder/temp.macosx-10.12-x86_64-3.5/Users/james/Dropbox/code/sysid-nets/si2_env/lib/python3.5/site-packages/mujoco_py/cymj.o -fopenmp -w


CompileError: command '/usr/local/bin/gcc-7' failed with exit status 1

In [ ]:
N_train = 30000
N_test = 500

DG = DataGenerator(config,env,'Hopper-v2')
Y,X = DG.sample_trajectories(train_hopper_pol,400,N_train,return_lists=False)
Y_test,X_test = DG.sample_trajectories(test_hopper_pol,400,N_test,return_lists=False)
with open('hopper_data_long.npz', 'wb') as f:
    np.savez(f, Y=Y, X=X, Y_test=Y_test, X_test=X_test)

In [ ]:
with open('hopper_data.npz', 'rb') as f:
    data = np.load(f)

    X = data['X']
    Y = data['Y']
    Y_test = data['Y_test']
    X_test = data['X_test']

## DynamicsAgent

In [ ]:
g3 = tf.Graph()
sess3 = tf.Session(config=tf.ConfigProto(log_device_placement=True), graph=g3)
dyn_model = AdaptiveDynamics(config)
dyn_model.construct_model(sess3,g3)

In [ ]:
dyn_model.train(sess3, Y,X, 50000)

In [ ]:
ind =7
import matplotlib.pyplot as plt
plt.figure(figsize=(5,24))
T = X_test[ind,:,:].shape[0]
for i in range(config['x_dim']):
    plt.subplot(config['x_dim'],1,i+1)
    plt.plot(np.arange(T), X_test[ind,:,i])
    plt.ylabel(r"$x_"+str(i+1)+"(t)$")
plt.show()

In [ ]:
ind = 4
T = 100
N_samples = 50
t_init = 50

fig = plt.figure(figsize=(9,7))
for i,Nu in enumerate( [0, 28, 56, 84] ):
    test_adaptive_dynamics(dyn_model, sess3, X_test[ind:ind+1,t_init:t_init+T,:], Y_test[ind:ind+1,t_init:t_init+T,:], N_samples, Nu, T_rollout=15)

plt.tight_layout(w_pad=0.2,h_pad=0.2)
plt.savefig('figures/hopper_rollouts_2.pdf')
plt.show()

In [ ]:
ind = 8
sample_size_list = [0,5,10,15]
T=50
N_samples=50
T_rollout = 35

plt.figure(figsize=(9,5))
for i,Nu in enumerate(sample_size_list):
    title=None
    legend=False
    if i == 0:
        legend=True
        title=True
        
    ax1 = plt.subplot(len(sample_size_list),3,3*i+1)
    gen_pendulum_sample_fig(agent, sess1, X_test[ind:ind+1,:,:], Y_test[ind:ind+1,:,:], Nu, N_samples=N_samples,T=T, T_rollout=T_rollout)
    if i == 0:
        plt.title('ALPaCA')
    if i < len(sample_size_list) - 1:
        plt.setp(ax1.get_xticklabels(), visible=False)
    
    ax2 = plt.subplot(len(sample_size_list),3,3*i+2, sharey=ax1)
    gen_pendulum_sample_fig(agent_nometa, sess2, X_test[ind:ind+1,:,:], Y_test[ind:ind+1,:,:], Nu, N_samples=N_samples,T=T, T_rollout=T_rollout)
    plt.setp(ax2.get_yticklabels(), visible=False)
    if i == 0:
        plt.title('ALPaCA (no meta)')
    if i < len(sample_size_list) - 1:
        plt.setp(ax2.get_xticklabels(), visible=False)
    
    ax3 = plt.subplot(len(sample_size_list),3,3*i+3, sharey=ax1)
    #gen_pendulum_sample_fig_gp(X_test[ind:ind+1,:,:], Y_test[ind:ind+1,:,:], Nu, N_samples=N_samples,T=T, T_rollout=T_rollout)
    plt.setp(ax3.get_yticklabels(), visible=False)
    if i == 0:
        plt.title('GPR')
    if i < len(sample_size_list) - 1:
        plt.setp(ax3.get_xticklabels(), visible=False)

plt.tight_layout(w_pad=0.0,h_pad=0.2)
plt.savefig('figures/pendulum_three.pdf')
plt.show()